# Simple LangChain Agent

The program is a simple LangChain agent that uses Neo4j.

The code:

- Creates an `OpenAI` LLM `model`
- Connects to your Neo4j database
- Defines a `Get-graph-database-schema` tool
- Creates a `react` (Reasoning and Acting) agent using the `model` and `tools`
- Streams the agent's response

Review and run the code and answer the following questions: 

1. What is the agent's function? 
2. What can it do?
3. How could you extend it?

***

Load the environment variables and import the required Python modules.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_neo4j import Neo4jGraph

Create a `gpt-4o` LLM `model` and connect to your Neo4j `graph` database.

In [2]:
# Initialize the LLM
model = init_chat_model("gpt-4o", model_provider="openai")

# Connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
)

Create the `tools` your agent will use.

In [3]:
# Define functions for each tool in the agent

@tool("Get-graph-database-schema")
def get_schema():
    """Get the schema of the graph database."""
    context = graph.schema
    return context

# Define a list of tools for the agent
tools = [get_schema]

> The agent will use the tool's name (`Get-graph-database-schema`), and docstring (`Get the schema of the graph database.`) to determine whether it should execute the tool to resolve a user's query.

***

Create the agent using the LLM `model` and `tools`.

In [4]:
agent = create_react_agent(
    model, 
    tools
)

Create a query for the agent, pass it to the agent, and stream the messages created by the agent.

When you run the agent, you will see:

1. The messages between `Human`, `AI`, and `Tool`
2. The context of the database schema
3. The agent's final response

In [5]:
# Run the application
query = "Summarise the schema of the graph database."

for step in agent.stream(
    {
        "messages": [{"role": "user", "content": query}]
    },
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Summarise the schema of the graph database.
================================== Ai Message ==================================
Tool Calls:
  Get-graph-database-schema (call_S8C58Dm0T9A35pbXEPyPx8od)
 Call ID: call_S8C58Dm0T9A35pbXEPyPx8od
  Args:
================================= Tool Message =================================
Name: Get-graph-database-schema

Node properties:
Chunk {:ID: STRING, embedding: LIST, text: STRING}
AssetManager {:ID: STRING, managerName: STRING}
Company {:ID: STRING, name: STRING, ticker: STRING}
Document {:ID: STRING, path: STRING}
Executive {:ID: STRING, name: STRING}
FinancialMetric {:ID: STRING, name: STRING}
Product {:ID: STRING, name: STRING}
RiskFactor {:ID: STRING, name: STRING}
StockType {:ID: STRING, name: STRING}
TimePeriod {:ID: STRING, name: STRING}
Transaction {:ID: STRING, name: STRING}
Relationship properties:
OWNS {position_status: STRING}
The relationships:
(:Chu

Experiment with agent, modify the `query` to ask different questions, for example:

* `"How are Products related to other entities?"`
* `"What questions can I answer using this graph database?"`
* `"How does the graph model relate financial documents to risk factors?"`

***

[View the complete code](solutions/02_01_simple_agent.py)

In [7]:
# Run the application
query = "What questions can I answer using this graph database?"

for step in agent.stream(
    {
        "messages": [{"role": "user", "content": query}]
    },
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What questions can I answer using this graph database?
================================== Ai Message ==================================
Tool Calls:
  Get-graph-database-schema (call_WDQ46yskR4rFuKMy8pWqlvde)
 Call ID: call_WDQ46yskR4rFuKMy8pWqlvde
  Args:
================================= Tool Message =================================
Name: Get-graph-database-schema

Node properties:
Chunk {:ID: STRING, embedding: LIST, text: STRING}
AssetManager {:ID: STRING, managerName: STRING}
Company {:ID: STRING, name: STRING, ticker: STRING}
Document {:ID: STRING, path: STRING}
Executive {:ID: STRING, name: STRING}
FinancialMetric {:ID: STRING, name: STRING}
Product {:ID: STRING, name: STRING}
RiskFactor {:ID: STRING, name: STRING}
StockType {:ID: STRING, name: STRING}
TimePeriod {:ID: STRING, name: STRING}
Transaction {:ID: STRING, name: STRING}
Relationship properties:
OWNS {position_status: STRING}
The relations